# MTech Data Warehouse Scripts

This is the list of scripts run on the sqlmtech, Aviagen_Chicken_V7Live database and imported into the MTech Data Warehouse dashboard


## Entity

This table creates a list of complex entity numbers and earliest transaction as StartDate and most recent transaction as EndDate. This table is used to determine the range of dates to create to convert the transactional data to daily data. 

In [1]:
DECLARE @startdate datetime 
SET @startdate = '1/1/16'

;with inventory_dates as 
(
    SELECT rtrim(b.FarmNo) + '-' + rtrim(b.EntityNo) AS ComplexEntityNo
    ,min(xDate) AS StartDate
    ,max(xDate) AS EndDate
        -- ,CASE WHEN min(a.xDate) < min(d.xDate) THEN min(a.xDate) ELSE min(d.xDate) END As StartDate
        -- ,CASE WHEN Max(a.xDate) > Max(d.xDate) THEN Max(a.xDate) ELSE Max(d.xDate) END AS EndDate 
    FROM mtech.BimEntityInventory a 
    INNER JOIN mtech.mvProteinEntities b 
        ON a.ProteinEntitiesIRN = b.IRN
    INNER JOIN mtech.BimEntities c 
        ON a.ProteinEntitiesIRN = c.ProteinEntitiesIRN
    -- RIGHT JOIN mtech.BimFieldTrans d 
    --     ON a.ProteinEntitiesIRN = d.ProteinEntitiesIRN
    WHERE a.EntityHistoryFlag = 0
    AND AvgHatchDate >= @startdate
    AND a.xDate > '11/30/1899'
    Group by FarmNo, EntityNo
)

,trans_dates as 
(
    SELECT rtrim(b.FarmNo) + '-' + rtrim(b.EntityNo) AS ComplexEntityNo
    ,min(xDate) AS StartDate
    ,max(xDate) AS EndDate
        -- ,CASE WHEN min(a.xDate) < min(d.xDate) THEN min(a.xDate) ELSE min(d.xDate) END As StartDate
        -- ,CASE WHEN Max(a.xDate) > Max(d.xDate) THEN Max(a.xDate) ELSE Max(d.xDate) END AS EndDate 
    FROM mtech.BimFieldTrans a 
    INNER JOIN mtech.mvProteinEntities b 
        ON a.ProteinEntitiesIRN = b.IRN
    INNER JOIN mtech.BimEntities c 
        ON a.ProteinEntitiesIRN = c.ProteinEntitiesIRN
    WHERE a.EntityHistoryFlag = 0
    AND AvgHatchDate >= @startdate
    AND a.xDate > '11/30/1899'
    Group by FarmNo, EntityNo
)

select top 10 
a.ComplexEntityNo
,CASE WHEN a.StartDate < coalesce(b.StartDate, '11/30/3000') then a.StartDate else b.StartDate END AS StartDate
,CASE WHEN a.EndDate > coalesce(b.EndDate, '11/30/1899') THEN a.EndDate else b.EndDate END AS EndDate
FROM inventory_dates a 
LEFT JOIN trans_dates b 
ON a.ComplexEntityNo = b.ComplexEntityNo

(10 rows affected)

Total execution time: 00:00:03.807

ComplexEntityNo,StartDate,EndDate
001.N1-02116.344,2016-01-05 00:00:00.000,2016-06-13 00:00:00.000
001.N1-02116.AP95,2016-01-05 00:00:00.000,2016-06-13 00:00:00.000
001.N1-12117.AP95,2017-03-23 00:00:00.000,2017-08-30 00:00:00.000
001.N1-16118.344,2018-04-17 00:00:00.000,2018-09-24 00:00:00.000
001.N1-16118.AP95,2018-04-17 00:00:00.000,2018-09-24 00:00:00.000
001.N1-23119.344,2019-06-07 00:00:00.000,2019-11-14 00:00:00.000
001.N1-23119.AP95,2019-06-07 00:00:00.000,2019-11-14 00:00:00.000
001.N1-28220.AP95,2020-07-11 00:00:00.000,2020-10-12 00:00:00.000
001.N1-32116.AP95,2016-08-06 00:00:00.000,2017-01-13 00:00:00.000
001.N1-41117.344,2017-10-14 00:00:00.000,2018-03-23 00:00:00.000


## Entity Detail

This table contains a list of all of the entities in the database and the flock level detail such as average hatch date, date capped, birds placed, moved and capped

In [2]:
with flocks AS 
(
    SELECT
        rtrim(pe.FarmNo) + '-' + rtrim(pe.EntityNo) AS ComplexEntityNo
        ,rtrim(DivisionNo) AS DivisionNo 
        ,rtrim(DivisionName) AS DivisionName
        ,rtrim(pe.FarmNo) AS FarmNo 
        ,rtrim(pe.FarmName) AS FarmName
        ,rtrim(pe.EntityNo) AS EntityNo
        ,pe.BreedNo
        ,rtrim(pe.GrowoutNo) AS GrowoutNo
        ,rtrim(pe.GrowoutName) AS GrowoutName
        ,rtrim(pts.TechSupervisorName) AS TechSupervisorName
        ,COALESCE(
            CASE WHEN GenerationCode = 4 THEN 'GP' ELSE NULL END,
            CASE WHEN GenerationCode = 3 THEN 'GGP' ELSE NULL END, 
            CASE WHEN GenerationCode = 2 THEN 'GGGP' ELSE NULL END,
            CASE WHEN GenerationCode = 1 THEN 'Pedigree' ELSE NULL END,
            'OTHER') AS GenerationCode 
        ,AvgDateMovedIn 
        ,AvgDateMovedOut
        ,FirstDateMovedIn
        ,AvgDateSold
        ,AvgDatePlaced
        ,AvgHatchDate
        ,LastDateSold
        ,LastDateMovedOut
        ,GrowDateMoved
        ,CASE WHEN ([STATUS] = 1 AND AvgDateSold = '11/30/1899') THEN 'Active' ELSE 'Depleted' END AS [STATUS]
        ,be.ExternalChickSource
    FROM mtech.mvProteinEntities pe 
    LEFT JOIN mtech.ProteinTechSupervisors pts ON pe.ProteinTechSupervisorsIRN = pts.IRN
    LEFT JOIN mtech.BimEntities be ON be.ProteinEntitiesIRN = pe.IRN
    LEFT JOIN mtech.BimCapitalizationTrans ct ON ct.ProteinEntitiesIRN = pe.IRN
        AND ct.PostStatus = 2 
        AND ct.VoidFlag = 0
    LEFT JOIN mtech.mvProteinCostCenters pcc ON pcc.IRN = pe.ProteinCostCentersIRN
    LEFT JOIN mtech.ProteinDivisions pd ON pd.IRN = pcc.ProteinDivisionsIRN
    WHERE AvgHatchDate>='1/1/16' 
    and GRN = 'F' 
)

,placements AS 
(
SELECT rtrim(b.FarmNo) + '-' + rtrim(b.EntityNo) AS ComplexEntityNo
,SUM(CASE WHEN (SEX = 1 AND SourceCode = 'CHICK' AND EntityHistoryFlag = 0) THEN Quantity ELSE NULL END) AS FemalePlacement
,SUM(CASE WHEN (SEX = 2 AND SourceCode = 'Chick' AND EntityHistoryFlag = 0) THEN Quantity ELSE NULL END) AS MalePlacement
,SUM(CASE WHEN (SEX = 1 AND SourceCode = 'Pmts Transfers' AND TransCode=1) THEN Quantity ELSE NULL END) AS FemaleMovement
,SUM(CASE WHEN (SEX = 2 AND SourceCode = 'Pmts Transfers' AND TransCode=1) THEN Quantity ELSE NULL END) AS MaleMovement
FROM mtech.BimEntityInventory a 
INNER JOIN mtech.mvProteinEntities b 
    ON a.ProteinEntitiesIRN = b.IRN
GROUP BY FarmNo, EntityNo
)

,capped AS 
(
    select rtrim(FarmNo) + '-' + rtrim(EntityNo) AS ComplexEntityNo
    ,MAX(CASE WHEN GRN = 'F' THEN DateCap ELSE NULL END) AS DateCap
    ,SUM(CASE WHEN GRN != 'F' THEN HensCap ELSE NULL END) AS HensCap
    ,SUM(CASE WHEN GRN != 'M' THEN MalesCap ELSE NULL END) AS MalesCap
    from mtech.mvBimEntities 
    where DateCap is not null
    Group by FarmNo, EntityNo
)


Select top 10  
f.*, convert(int,FemalePlacement) FemalePlacement, convert(int, MalePlacement) MalePlacement, convert(int, FemaleMovement) FemaleMovement, convert(int, MaleMovement) MaleMovement
,c.DateCap, c.MalesCap, c.HensCap
,((CASE WHEN (DateCap > '18991130' AND DateCap <= '12/31/9999') OR ExternalChickSource = 1 THEN 3 
        WHEN (GrowDateMoved > '18991130' AND GrowDateMoved < '12/31/9999') THEN 2 ELSE 1 END)) AS EntityStage
FROM flocks f 
LEFT JOIN placements p 
    ON f.ComplexEntityNo = p.ComplexEntityNo
LEFT JOIN capped c 
    ON f.ComplexEntityNo = c.ComplexEntityNo

Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:00:07.069

ComplexEntityNo,DivisionNo,DivisionName,FarmNo,FarmName,EntityNo,BreedNo,GrowoutNo,GrowoutName,TechSupervisorName,GenerationCode,AvgDateMovedIn,AvgDateMovedOut,FirstDateMovedIn,AvgDateSold,AvgDatePlaced,AvgHatchDate,LastDateSold,LastDateMovedOut,GrowDateMoved,STATUS,ExternalChickSource,FemalePlacement,MalePlacement,FemaleMovement,MaleMovement,DateCap,MalesCap,HensCap,EntityStage
001.N1-32116.AP95,BR,Brazil,001.N1,Granja 1.2,32116.AP95,AP95,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2016-10-20 00:00:00.000,2016-11-10 00:00:00.000,2016-08-15 00:00:00.000,1899-11-30 00:00:00.000,2016-08-06 00:00:00.000,2016-08-05 00:00:00.000,1899-11-30 00:00:00.000,2017-01-13 00:00:00.000,1899-11-30 00:00:00.000,Depleted,0,8240,2069,22564,2461,NULL,NULL,NULL,1
001.N2-16118.AP95,BR,Brazil,001.N2,Granja 1.2,16118.AP95,AP95,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2018-09-24 00:00:00.000,1899-11-30 00:00:00.000,2018-09-24 00:00:00.000,2019-07-16 00:00:00.000,2018-04-17 00:00:00.000,2018-04-16 00:00:00.000,2019-07-16 00:00:00.000,1899-11-30 00:00:00.000,1899-11-30 00:00:00.000,Depleted,0,NULL,NULL,5836,625,2018-09-25 00:00:00.000,625,5836,3
001.N5-22120.AP95,BR,Brazil,001.N5,Granja 1.2,22120.AP95,AP95,BR,BRAZIL,Aldair Alves,GP,2020-07-09 00:00:00.000,2020-07-09 00:00:00.000,2020-06-06 00:00:00.000,1899-11-30 00:00:00.000,2020-05-29 00:00:00.000,2020-05-28 00:00:00.000,1899-11-30 00:00:00.000,2020-09-22 00:00:00.000,1899-11-30 00:00:00.000,Active,0,9519,3804,20820,4083,NULL,NULL,NULL,1
001.N6-14117.344,BR,Brazil,001.N6,Granja 1.2,14117.344,344,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2017-06-06 00:00:00.000,2017-07-04 00:00:00.000,2017-04-17 00:00:00.000,1899-11-30 00:00:00.000,2017-04-04 00:00:00.000,2017-04-03 00:00:00.000,1899-11-30 00:00:00.000,2017-09-11 00:00:00.000,1899-11-30 00:00:00.000,Depleted,0,2446,2086,4140,1869,NULL,NULL,NULL,1
001.N6-31119.AP95,BR,Brazil,001.N6,Granja 1.2,31119.AP95,AP95,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2019-10-04 00:00:00.000,2019-11-04 00:00:00.000,2019-08-15 00:00:00.000,1899-11-30 00:00:00.000,2019-08-02 00:00:00.000,2019-08-01 00:00:00.000,1899-11-30 00:00:00.000,2020-01-09 00:00:00.000,1899-11-30 00:00:00.000,Depleted,0,8563,2374,16360,2672,NULL,NULL,NULL,1
001.N6-37116.AP95,BR,Brazil,001.N6,Granja 1.2,37116.AP95,AP95,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2016-10-17 00:00:00.000,2016-11-18 00:00:00.000,2016-09-16 00:00:00.000,1899-11-30 00:00:00.000,2016-09-09 00:00:00.000,2016-09-08 00:00:00.000,1899-11-30 00:00:00.000,2017-02-16 00:00:00.000,1899-11-30 00:00:00.000,Depleted,0,9219,2397,25560,1120,NULL,NULL,NULL,1
001.N7-14117.AP95,BR,Brazil,001.N7,Granja 1.2,14117.AP95,AP95,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2017-09-11 00:00:00.000,1899-11-30 00:00:00.000,2017-09-11 00:00:00.000,2018-06-27 00:00:00.000,2017-04-04 00:00:00.000,2017-04-03 00:00:00.000,2018-06-27 00:00:00.000,1899-11-30 00:00:00.000,1899-11-30 00:00:00.000,Depleted,0,NULL,NULL,7067,715,2017-09-12 00:00:00.000,715,7067,3
001.N7-20118.AP95,BR,Brazil,001.N7,Granja 1.2,20118.AP95,AP95,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2018-10-22 00:00:00.000,1899-11-30 00:00:00.000,2018-10-22 00:00:00.000,2019-08-23 00:00:00.000,2018-05-15 00:00:00.000,2018-05-14 00:00:00.000,2019-08-23 00:00:00.000,1899-11-30 00:00:00.000,1899-11-30 00:00:00.000,Depleted,0,NULL,NULL,7889,797,2018-10-23 00:00:00.000,797,7889,3
004.N2-50117.344,BR,Brazil,004.N2,Granja 4.6,50117.344,344,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2018-05-24 00:00:00.000,2018-09-25 00:00:00.000,2018-05-23 00:00:00.000,2019-01-10 00:00:00.000,2017-12-14 00:00:00.000,2017-12-13 00:00:00.000,2019-01-10 00:00:00.000,2018-09-29 00:00:00.000,1899-11-30 00:00:00.000,Depleted,0,NULL,NULL,5404,532,2018-05-24 00:00:00.000,526,5363,3
004.N3-03B19.AP95,BR,Brazil,004.N3,Granja 4.6,03B19.AP95,AP95,BR,BRAZIL,Gilberto Rodrigues Costa,GP,2019-03-29 00:00:00.000,2019-04-26 00:00:00.000,2019-01-31 00:00:00.000,1899-11-30 00:00:00.000,2019-01-16 00:00:00.000,2019-01-15 00:00:00.000,1899-11-30 00:00:00.000,2019-06-25 00:00:00.00

## Transactions

This table contains every transaction associated with the set of Complex Entity numbers. 

In [3]:
;with entities as 
(
    SELECT ProteinEntitiesIRN
    ,rtrim(FarmNo) + '-' + rtrim(EntityNo) AS ComplexEntityNo
    ,(SELECT AvgHatchDate from mtech.mvBimEntities WHERE GRN='F' and FarmNo = a.FarmNo and EntityNo = a.EntityNo) AS AvgHatchDate
    ,(SELECT DateCap from mtech.mvBimEntities WHERE GRN='F' and FarmNo = a.FarmNo and EntityNo = a.EntityNo) AS DateCap
    FROM mtech.mvBimEntities a
    WHERE GRN IN ('H','P')
    AND AvgHatchDate >='1/1/16'
)

,hatch as 
(
SELECT rtrim(FarmNo) + '-' + rtrim(EntityNo) AS ComplexEntityNo
,ProdDate
,SUM(EggsSet) AS EggsSet
,SUM(TotalChicksHatched) AS TotalChicksHatched
FROM mtech.HimHatchTrans a 
join mtech.ProteinEntities b 
ON a.ProteinEntitiesIRN = b.IRN 
JOIN mtech.ProteinFarms c 
ON b.ProteinFarmsIRN = c.IRN
GROUP BY rtrim(FarmNo) + '-' + rtrim(EntityNo), ProdDate
)

,data as 
(
SELECT ComplexEntityNo
    , BFT.xDate
    ,datediff(week,AvgHatchDate, xDate) as Weeks
    ,DATEADD(dd, 7-(DATEPART(dw, xDate)), xDate) [WeekEnd]
    ,CASE WHEN xDate >= DateCap THEN 1 ELSE 0 END AS Capped
	 , SUM(BFT.AddedF) AddedF
	 , SUM(BFT.AddedM) AddedM
     , SUM(CullEggsProd) CullEggsProd
     , SUM(HatchEggsProd) HatchEggsProd
     , SUM(U_HatchEgg) U_HatchEgg
     , SUM(TotalEggsProd) TotalEggsProd
	 , SUM(BFT.U_HatchEggProd) U_HatchEggProd
	 , SUM(BFT.U_HEGradeAProd) U_HEGradeAProd
	 , SUM(BFT.U_HEGradeBProd) U_HEGradeBProd
	 , SUM(BFT.U_HECategory4Prod) U_HECategory4Prod
	 , SUM(BFT.U_HECategory5Prod) U_HECategory5Prod
	 , SUM(BFT.U_HECategory6Prod) U_HECategory6Prod
	 , SUM(BFT.U_HECategory7Prod) U_HECategory7Prod
	 , SUM(BFT.U_HECategory8Prod) U_HECategory8Prod
	 , SUM(BFT.U_HECategory9Prod) U_HECategory9Prod
	 , SUM(BFT.U_CullEggFloorProd) U_CullEggFloorProd
	 , SUM(BFT.U_CullEggSoftShellProd) U_CullEggSoftShellProd
	 , SUM(BFT.U_CDoubleYolkProd) U_CDoubleYolkProd
	 , SUM(BFT.U_CMisshapedEggProd) U_CMisshapedEggProd
	 , SUM(BFT.U_CSmallEggProd) U_CSmallEggProd
     ,0 AS U_CDirtyProd
	 , SUM(BFT.U_UnusedEggsProd) U_UnusedEggsProd
     ,0 AS U_HE2ndNWProd
     ,0 AS U_HE2ndWProd
	 , SUM(BFT.U_CDestroyedProd) U_CDestroyedProd
	 , SUM(BFT.U_ComercialProd) U_CommercialProd
	 , SUM(BFT.U_CCrackedEggProd) U_CCrackedEggProd
	 , SUM(BFT.U_COtherProd) U_COtherProd
	 , CASE WHEN SUM(BFT.EggWeight * TotalEggsProd) > 0 THEN SUM(BFT.EggWeight * TotalEggsProd)/SUM(CASE WHEN EggWeight > 0 THEN TotalEggsProd ELSE 0 END) ELSE 0 END AS EggWeight
	 , SUM(BFT.U_FloorEggs) U_FloorEggs
	 , SUM(BFT.FeedConsumedM) FeedConsumedM
	 , SUM(BFT.FeedConsumedF) FeedConsumedF
	 , SUM(BFT.MortalityM) MortalityM
	 , SUM(BFT.U_CLegMale) U_CLegMale
	 , SUM(BFT.U_CRupturedTendonMale) U_CRupturedTendonMale
	 , SUM(BFT.U_CullOtherMale) U_CullOtherMale
	 , SUM(BFT.U_CullSelectionMale) U_CullSelectionMale
	 , SUM(BFT.U_CullSexErrorMale) U_CullSexErrorMale
	 , SUM(BFT.MortalityF) MortalityF
	 , SUM(BFT.U_CLeg) U_CLeg
	 , SUM(BFT.U_CRupturedTendon) U_CRupturedTendon
	 , SUM(BFT.U_CullOther) U_CullOther
	 , SUM(BFT.U_CullSelection) U_CullSelection
	 , SUM(BFT.U_CullSexError) U_CullSexError
	 , Avg(BFT.WeightF) WeightF
	 , Avg(BFT.WeightM) WeightM
	 , SUM(BFT.UniformityF) UniformityF
	 , SUM(BFT.UniformityM) UniformityM
	 , Avg(BFT.UniformityCvF) UniformityCvF
	 , Avg(BFT.UniformityCvM) UniformityCvM
	 , SUM(BFT.Water) Water
	 , SUM(BFT.U_CullDropCrop) U_CullDropCrop
	 , SUM(BFT.U_CullDropCropMale) U_CullDropCropMale
	 , SUM(BFT.U_CFlockSample) U_CFlockSample
	 , SUM(BFT.U_CFlockSampleMale) U_CFlockSampleMale
     ,SUM(coalesce(U_CullOther,0) + coalesce(U_CullSelection,0) + coalesce(U_CullSexError,0) + coalesce(U_CLeg,0) + coalesce(U_CRupturedTendon,0) + coalesce(U_CullDropCrop,0) + coalesce(U_CFlockSample,0)) CullF
     ,SUM(coalesce(U_CullOtherMale,0) + coalesce(U_CullSelectionMale,0) + coalesce(U_CullSexErrorMale,0) + coalesce(U_CLegMale,0) + coalesce(U_CRupturedTendonMale,0) + coalesce(U_CullDropCropMale,0)+ coalesce(U_CFlockSampleMale,0)) AS CullM
     ,SUM(coalesce(MortalityF,0) + coalesce(U_CullOther,0) + coalesce(U_CullSelection,0) + coalesce(U_CullSexError,0) + coalesce(U_CLeg,0) + coalesce(U_CRupturedTendon,0) + coalesce(U_CullDropCrop,0)+ coalesce(U_CFlockSample,0)) DepletionF
     ,SUM(coalesce(MortalityM,0) + coalesce(U_CullOtherMale,0) + coalesce(U_CullSelectionMale,0) + coalesce(U_CullSexErrorMale,0) + coalesce(U_CLegMale,0) + coalesce(U_CRupturedTendonMale,0) + coalesce(U_CullDropCropMale,0)+ coalesce(U_CFlockSampleMale,0)) AS DepletionM
  FROM entities VBE
 INNER JOIN mtech.BimFieldTrans BFT
    ON VBE.ProteinEntitiesIRN = BFT.ProteinEntitiesIRN
 WHERE BFT.EntityHistoryFlag = 0
 GROUP BY VBE.ComplexEntityNo, BFT.xDate, AvgHatchDate, DateCap
)

 Select top 10
 a.*
 , b.EggsSet, TotalChicksHatched
 FROM data a 
 LEFT JOIN hatch b 
 ON a.ComplexEntityNo = b.ComplexEntityNo and a.xDate=b.ProdDate
 ORDER BY xDate

Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:01:25.123

ComplexEntityNo,xDate,Weeks,WeekEnd,Capped,AddedF,AddedM,CullEggsProd,HatchEggsProd,U_HatchEgg,TotalEggsProd,U_HatchEggProd,U_HEGradeAProd,U_HEGradeBProd,U_HECategory4Prod,U_HECategory5Prod,U_HECategory6Prod,U_HECategory7Prod,U_HECategory8Prod,U_HECategory9Prod,U_CullEggFloorProd,U_CullEggSoftShellProd,U_CDoubleYolkProd,U_CMisshapedEggProd,U_CSmallEggProd,U_CDirtyProd,U_UnusedEggsProd,U_HE2ndNWProd,U_HE2ndWProd,U_CDestroyedProd,U_CommercialProd,U_CCrackedEggProd,U_COtherProd,EggWeight,U_FloorEggs,FeedConsumedM,FeedConsumedF,MortalityM,U_CLegMale,U_CRupturedTendonMale,U_CullOtherMale,U_CullSelectionMale,U_CullSexErrorMale,MortalityF,U_CLeg,U_CRupturedTendon,U_CullOther,U_CullSelection,U_CullSexError,WeightF,WeightM,UniformityF,UniformityM,UniformityCvF,UniformityCvM,Water,U_CullDropCrop,U_CullDropCropMale,U_CFlockSample,U_CFlockSampleMale,CullF,CullM,DepletionF,DepletionM,EggsSet,TotalChicksHatched
315-3081892YC,2000-01-30 00:00:00.000,-957,2000-02-05 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-01-31 00:00:00.000,-957,2000-02-05 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-02-01 00:00:00.000,-957,2000-02-05 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-02-02 00:00:00.000,-957,2000-02-05 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-02-03 00:00:00.000,-957,2000-02-05 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-02-04 00:00:00.000,-957,2000-02-05 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-02-05 00:00:00.000,-957,2000-02-05 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-02-06 00:00:00.000,-956,2000-02-12 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-02-07 00:00:00.000,-956,2000-02-12 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,NULL
315-3081892YC,2000-02-08 00:00:00.000,-956,2000-02-12 00:00:00.000,0,0,0,0,0,NULL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000000,NULL,0.0000000000,0.0000000000,0,0,0,0,0,0,0,0,NULL,0,0,0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.00

## Inventory

This table includes every inventory transaction for every complex entity number in the set

In [5]:
SELECT TOP 10
    rtrim(b.FarmNo) + '-' + rtrim(b.EntityNo) AS ComplexEntityNo
    ,xDate 
    ,SUM(CASE WHEN SEX = 1 THEN Quantity ELSE 0 END) AS FemaleInvTransactions
    ,SUM(CASE WHEN SEX = 2 THEN Quantity ELSE 0 END) AS MaleInvTransactions
FROM mtech.BimEntityInventory a 
INNER JOIN mtech.mvProteinEntities b 
    ON a.ProteinEntitiesIRN = b.IRN
INNER JOIN mtech.BimEntities c 
    ON a.ProteinEntitiesIRN = c.ProteinEntitiesIRN
WHERE a.EntityHistoryFlag = 0
AND AvgHatchDate > '1/1/16'
GROUP BY FarmNo, EntityNo,xDate
ORDER BY ComplexEntityNo, xDate

(10 rows affected)

Total execution time: 00:00:08.122

ComplexEntityNo,xDate,FemaleInvTransactions,MaleInvTransactions
001.N1-02116.344,1899-11-30 00:00:00.000,0.0000000000,0.0000000000
001.N1-02116.344,2016-01-05 00:00:00.000,1440.0000000000,1439.0000000000
001.N1-02116.344,2016-01-06 00:00:00.000,-2.0000000000,-1.0000000000
001.N1-02116.344,2016-01-07 00:00:00.000,-3.0000000000,-2.0000000000
001.N1-02116.344,2016-01-08 00:00:00.000,-1.0000000000,0.0000000000
001.N1-02116.344,2016-01-09 00:00:00.000,0.0000000000,-1.0000000000
001.N1-02116.344,2016-01-10 00:00:00.000,-2.0000000000,-3.0000000000
001.N1-02116.344,2016-01-11 00:00:00.000,-12.0000000000,-12.0000000000
001.N1-02116.344,2016-01-12 00:00:00.000,-1.0000000000,-1.0000000000
001.N1-02116.344,2016-01-14 00:00:00.000,0.0000000000,-2.0000000000
